#  **"Census Data" Data Mining and Analysys**
This Dataset used for this assignment is the Census Dataset for predicting whether annual income of an individual exceeds $50K/yr <br> from: https://archive.ics.uci.edu/dataset/2/adult


### This dataset has the following problems identified:
<ul>
    <li> No column names</li>
    <li> The Missing values are labeled as ' ?'</li>
    <li> Duplicate Rows</li>
</ul>

---

## **1. Initial exploration**

>**The following cell blocks are executed for initial exploration of the dataset, its missing and duplicated values, shapes, and statistics**

In [130]:

import pandas as pd
import numpy as np

>**The dataset doesn't have column names by default, so we define the column names before we import the dataset so we can read the data properly**



In [131]:
# Define column names

columns = [
    "age", "workclass", "fnlwgt", "education", "education-num",
    "marital-status", "occupation", "relationship", "race", "sex",
    "capital-gain", "capital-loss", "hours-per-week", "native-country", "income"
]

>**In this dataset, the null values are marked as " ?". To ensure we read " ?" as empty, we use na_values to treat " ?" as NaN.** 

*The cleaning process later will include the handling of " ?" to NaN in the cleaned dataset.*

In [132]:
# Load dataset
census_df = pd.read_csv(
    "../data/raw_dataset.csv",
    header=None,    
    names=columns,        # assign the column names
    na_values=' ?'         # treat ' ?' as missing values
)

In [133]:
# Display basic info

print("=== Dataframe Info ===")
print(census_df.info())
print("\n")

=== Dataframe Info ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       30725 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education-num   32561 non-null  int64 
 5   marital-status  32561 non-null  object
 6   occupation      30718 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital-gain    32561 non-null  int64 
 11  capital-loss    32561 non-null  int64 
 12  hours-per-week  32561 non-null  int64 
 13  native-country  31978 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB
None




In [134]:
# Statistical description

print("=== Dataframe Description ===")
print(census_df.describe(include='all'))  # include='all' covers categorical columns too
print("\n")


=== Dataframe Description ===
                 age workclass        fnlwgt education  education-num  \
count   32561.000000     30725  3.256100e+04     32561   32561.000000   
unique           NaN         8           NaN        16            NaN   
top              NaN   Private           NaN   HS-grad            NaN   
freq             NaN     22696           NaN     10501            NaN   
mean       38.581647       NaN  1.897784e+05       NaN      10.080679   
std        13.640433       NaN  1.055500e+05       NaN       2.572720   
min        17.000000       NaN  1.228500e+04       NaN       1.000000   
25%        28.000000       NaN  1.178270e+05       NaN       9.000000   
50%        37.000000       NaN  1.783560e+05       NaN      10.000000   
75%        48.000000       NaN  2.370510e+05       NaN      12.000000   
max        90.000000       NaN  1.484705e+06       NaN      16.000000   

             marital-status       occupation relationship    race    sex  \
count            

In [135]:
# Missing values

print("=== Missing Values ===")
print(census_df.isnull().sum())
print("\n")

=== Missing Values ===
age                  0
workclass         1836
fnlwgt               0
education            0
education-num        0
marital-status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital-gain         0
capital-loss         0
hours-per-week       0
native-country     583
income               0
dtype: int64




In [136]:
# Check duplicates

print("=== Duplicate Rows ===")
print(census_df.duplicated().sum())
print("\n")

=== Duplicate Rows ===
24




## **2. Data Cleaning Process**

>**The following cell blocks are executed to handle missing values, standardize formats, and detect and treat outliers.**

In [137]:
# Handling Missing Values

# For categorical columns: fill NaN with mode (most frequent value)
categorical_cols = census_df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    mode_val = census_df[col].mode()[0]
    census_df[col] = census_df[col].fillna(mode_val)

# For numeric columns: fill NaN with median (robust to outliers)
numeric_cols = census_df.select_dtypes(include=['int64', 'float64']).columns
for col in numeric_cols:
    median_val = census_df[col].median()
    census_df[col] = census_df[col].fillna(median_val)

In [138]:
# Standardize Formats

# Strip extra spaces and lowercase for categorical consistency
for col in categorical_cols:
    census_df[col] = census_df[col].str.strip().str.lower()

# Standardize target variable to binary format (<=50k, >50k)
census_df["income"] = census_df["income"].replace({"<=50k": "0", ">50k": "1"})

In [139]:
# Remove duplicates

census_df = census_df.drop_duplicates()

In [140]:
# Detect and Treat Outliers (IQR method for numeric columns)

for col in numeric_cols:
    Q1 = census_df[col].quantile(0.25)
    Q3 = census_df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Cap outliers instead of dropping (winsorization)
    census_df[col] = np.where(census_df[col] < lower_bound, lower_bound, census_df[col])
    census_df[col] = np.where(census_df[col] > upper_bound, upper_bound, census_df[col])

In [141]:
# Save Cleaned Dataset

census_df.to_csv("../data/cleaned_dataset.csv", index=False)

# Display the cleaned DataFrame
census_df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39.0,state-gov,77516.0,bachelors,13.0,never-married,adm-clerical,not-in-family,white,male,0.0,0.0,40.0,united-states,0
1,50.0,self-emp-not-inc,83311.0,bachelors,13.0,married-civ-spouse,exec-managerial,husband,white,male,0.0,0.0,32.5,united-states,0
2,38.0,private,215646.0,hs-grad,9.0,divorced,handlers-cleaners,not-in-family,white,male,0.0,0.0,40.0,united-states,0
3,53.0,private,234721.0,11th,7.0,married-civ-spouse,handlers-cleaners,husband,black,male,0.0,0.0,40.0,united-states,0
4,28.0,private,338409.0,bachelors,13.0,married-civ-spouse,prof-specialty,wife,black,female,0.0,0.0,40.0,cuba,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27.0,private,257302.0,assoc-acdm,12.0,married-civ-spouse,tech-support,wife,white,female,0.0,0.0,38.0,united-states,0
32557,40.0,private,154374.0,hs-grad,9.0,married-civ-spouse,machine-op-inspct,husband,white,male,0.0,0.0,40.0,united-states,1
32558,58.0,private,151910.0,hs-grad,9.0,widowed,adm-clerical,unmarried,white,female,0.0,0.0,40.0,united-states,0
32559,22.0,private,201490.0,hs-grad,9.0,never-married,adm-clerical,own-child,white,male,0.0,0.0,32.5,united-states,0


## **3. Show Dataset Shapes, Statistics, and Sample Rows**

>**The following cell blocks are executed to show shapes, statistics, and sample rows.**

In [142]:
# Show the sample rows

print("=== Sample Rows ===")
print(census_df.sample(10))  
print("\n")

=== Sample Rows ===
        age     workclass    fnlwgt     education  education-num  \
3861   51.0     local-gov  175750.0       hs-grad            9.0   
28410  23.0       private  185106.0       hs-grad            9.0   
16725  20.0       private  163911.0  some-college           10.0   
740    19.0       private  192453.0  some-college           10.0   
23056  50.0   federal-gov  306707.0       hs-grad            9.0   
16967  35.0       private  100375.0  some-college           10.0   
12693  51.0     state-gov  155594.0  some-college           10.0   
1623   44.0       private  147206.0  some-college           10.0   
21082  29.0       private  415742.0  some-college           10.0   
13944  38.0  self-emp-inc  187411.0       hs-grad            9.0   

              marital-status         occupation    relationship   race  \
3861                divorced   transport-moving       unmarried  black   
28410          never-married  handlers-cleaners       own-child  black   
16725    

In [143]:
#Show the shape of the dataset

print("Dataset shape:", census_df.shape)

Dataset shape: (32537, 15)


>The code below shows the top labels for non-numerical attributes of the dataset

In [144]:
# Summary for categorical columns only

print("\n=== Categorical Summary ===")
print(census_df.describe(include=['object']))


=== Categorical Summary ===
       workclass education      marital-status      occupation relationship  \
count      32537     32537               32537           32537        32537   
unique         8        16                   7              14            6   
top      private   hs-grad  married-civ-spouse  prof-specialty      husband   
freq       24509     10494               14970            5979        13187   

         race    sex native-country income  
count   32537  32537          32537  32537  
unique      5      2             41      2  
top     white   male  united-states      0  
freq    27795  21775          29735  24698  


>The code below was based from Lesson003_Descriptive Statistics colab code (Iris Dataset) 
<br>to get calculate the summary statistics of the dataset's numerical attributes

In [145]:
from pandas.api.types import is_numeric_dtype


# Initialize an empty DataFrame to store the statistics
summary_df = pd.DataFrame(columns=['Column', 'Mean', 'Mode', 'Std', 'Min', 'Max'])

summary_frames = []

# Loop through the numeric columns and calculate statistics
for col in census_df.columns:
    if is_numeric_dtype(census_df[col]):
        mean = census_df[col].mean()
        std = census_df[col].std()
        min_val = census_df[col].min()
        max_val = census_df[col].max()

        mode_values = census_df[col].mode().values

        if len(mode_values) > 1:
            mode = np.array(mode_values)
        else:
            mode = mode_values[0]

        median = census_df[col].median()

        # Create a DataFrame for the current column
        col_summary = pd.DataFrame({'Column': [col], 'Mean': [mean], 'Mode': [mode],  
                                    'Median': [median], 'Standard Deviation': [std], 'Min': [min_val], 'Max': [max_val]})
        summary_frames.append(col_summary)

# Concatenate the list of DataFrames into one summary DataFrame
summary_df = pd.concat(summary_frames, ignore_index=True)

# Display the summary DataFrame
summary_df

,Column,Mean,Mode,Median,Standard Deviation,Min,Max
0,age,38.559855,36.0,37.0,13.554847,17.0,78.0
1,fnlwgt,186824.961736,415742.0,178356.0,95118.115529,12285.0,415742.0
2,education-num,10.125165,9.0,10.0,2.459436,4.5,16.0
3,capital-gain,0.000000,0.0,0.0,0.000000,0.0,0.0
4,capital-loss,0.000000,0.0,0.0,0.000000,0.0,0.0
5,hours-per-week,41.203246,40.0,40.0,6.187352,32.5,52.5
